# Data Loading

In [1]:
from __future__ import print_function, division
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import os
import numpy as np

# defining customized Dataset class for Udacity

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class UdacityDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, select_camera=None, slice_frames=None, select_ratio=1.0, select_range=None):
        
        assert select_ratio >= -1.0 and select_ratio <= 1.0 # positive: select to ratio from beginning, negative: select to ration counting from the end
        
        camera_csv = pd.read_csv(csv_file)
        csv_len = len(camera_csv)
        assert select_camera in ['left_camera', 'right_camera', 'center_camera'], "Invalid camera: {}".format(select_camera)
        
        if slice_frames:
            csv_selected = camera_csv[0:0] # empty dataframe
            for start_idx in range(0, csv_len, slice_frames):
                if select_ratio > 0:
                    end_idx = int(start_idx + slice_frames * select_ratio)
                else:
                    start_idx, end_idx = int(start_idx + slice_frames * (1 + select_ratio)), start_idx + slice_frames

                if end_idx > csv_len:
                    end_idx = csv_len
                if start_idx > csv_len:
                    start_idx = csv_len
                csv_selected = csv_selected.append(camera_csv[start_idx:end_idx])
            self.camera_csv = csv_selected
        elif select_range:
            csv_selected = camera_csv.iloc[select_range[0]: select_range[1]]
            self.camera_csv = csv_selected
        else:
            self.camera_csv = camera_csv
            
        self.root_dir = root_dir
        self.transform = transform
        
        # Keep track of mean and cov value in each channel
        self.mean = {}
        self.std = {}
        for key in ['angle', 'torque', 'speed']:
            self.mean[key] = np.mean(camera_csv[key])
            self.std[key] = np.std(camera_csv[key])
    
    def __len__(self):
        return len(self.camera_csv)
    
    def read_data_single(self, idx):
        path = os.path.join(self.root_dir, self.camera_csv['filename'].iloc[idx])
        image = io.imread(path)
        timestamp = self.camera_csv['timestamp'].iloc[idx]
        frame_id = self.camera_csv['frame_id'].iloc[idx]
        angle = self.camera_csv['angle'].iloc[idx]
        torque = self.camera_csv['torque'].iloc[idx]
        speed = self.camera_csv['speed'].iloc[idx]
        
        if self.transform:
            image_transformed = self.transform(image)
            del image
            image = image_transformed
        angle_t = torch.tensor(angle)
        torque_t = torch.tensor(torque)
        speed_t = torch.tensor(speed)
        del angle, torque, speed
            
        return image, timestamp, frame_id, angle_t, torque_t, speed_t
    
    def read_data(self, idx):
        if isinstance(idx, list):
            data = None
            for i in idx:
                new_data = self.read_data(i)
                if data is None:
                    data = [[] for _ in range(len(new_data))]
                for i, d in enumerate(new_data):
                    data[i].append(new_data[i])
                del new_data
                
            for stack_idx in [0, 3, 4, 5]: # we don't stack timestamp and frame_id since those are string data
                data[stack_idx] = torch.stack(data[stack_idx])
            
            return data
        
        else:
            return self.read_data_single(idx)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        data = self.read_data(idx)
        
        sample = {'image': data[0],
                  'timestamp': data[1],
                  'frame_id': data[2],
                  'angle': data[3],
                  'torque': data[4],
                  'speed': data[5]}
        
        del data
        
        return sample

In [2]:
# generate Batch with consecutive frames taken from input data

from torch.utils.data import Sampler
import random

class ConsecutiveBatchSampler(Sampler):
    
    def __init__(self, data_source, batch_size, seq_len, drop_last=False, shuffle=True, use_all_frames=False):
        r""" Sampler to generate consecutive Batches
        
        Args:
            data_source: Source of data
            batch_size: Size of batch
            seq_len: Number of frames in each sequence (used for context for prediction)
            drop: Wether to drop the last incomplete batch
            shuffle: Wether to shuffle the data
        Return:
            List of iterators, size: [batch_size x seq_len x n_channels x height x width]
        """
        super(ConsecutiveBatchSampler, self).__init__(data_source)
        
        self.data_source = data_source
        
        assert seq_len >= 1, "Invalid batch size: {}".format(seq_len)
        self.seq_len = seq_len
        self.drop_last = drop_last
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.use_all_frames_ = use_all_frames
    
    def __iter__(self):
        
        data_size = len(self.data_source)
        
        if self.use_all_frames_:
            start_indices = list(range(data_size))
        else:
            start_indices = list(range(1, data_size, self.seq_len))
            
        if self.shuffle:
            random.shuffle(start_indices)
        
        batch = []
        for idx, ind in enumerate(start_indices):
            if data_size - idx < self.batch_size and self.drop_last: # if last batch
                break
                
            seq = []
            if ind + 1 < self.seq_len:
                seq.extend([0]*(self.seq_len - ind - 1) + list(range(0, ind+1)))
            else:
                seq.extend(list(range(ind-self.seq_len+1, ind+1)))
            
            batch.append(seq)
            
            if len(batch) == self.batch_size or idx == data_size - 1:
                yield batch
                batch = []

    
    def __len__(self):
        length = len(self.data_source)
        batch_size = self.batch_size
        
        if length % batch_size == 0 or self.drop_last:
            return length // batch_size
        
        return length // batch_size + 1

In [3]:
def show_sample(sample):
    r""" Helper function for (batch) sample visualization
    
    Args:
        sample: Dictionary
    """
    image_dims = len(sample['image'].shape)
    assert image_dims <= 5, "Unsupported image shape: {}".format(sample['image'].shape)
    if image_dims == 3:
        plt.imshow(sample['image'])
    else:
        n0 = sample['image'].shape[0]
        n1 = sample['image'].shape[1] if image_dims == 5 else 1
        images_flattened = torch.flatten(sample['image'], end_dim=-4)
        fig, ax = plt.subplots(n0, n1, figsize=(25, 15))
        for i1 in range(n1):
            for i0 in range(n0):
                image = images_flattened[i0 * n1 + i1]
                axis = ax[i0, i1]
                axis.imshow(image.permute(1,2,0))
                axis.axis('off')
                axis.set_title("t={}".format(sample['timestamp'][i0][i1]))
                axis.text(10, 30, sample['frame_id'][i0][i1], color='red')
            
    


# Build the model

## 3D CNN with residual connection

In [4]:
# Build the model

import torch.nn as nn

# helper function to determine dimension after convolution
def conv_output_shape(in_dimension, kernel_size, stride):
    output_dim = []
    for (in_dim, kern_size, strd) in zip(in_dimension, kernel_size, stride):
        len = int(float(in_dim - kern_size) / strd + 1.)
        output_dim.append(len)
    
    return output_dim

        
class TemporalCNN(nn.Module):
    
    def _conv_unit(self, in_channels, out_channels, in_shape, kernel_size, stride, dropout_prob):
        r""" Return one 3D convolution unit
        
        Args:
            in_channels: Input channels of the Conv3D module
            out_channels: Output channels of the Conv3D module
            in_shape: Shape of the input image. i.e. The last 3 dimensions of the input tensor: D x H x W
            kernel_size: Kernel size
            stride: Stride
            dropout_prob: Probability of dropout layer
                         
        Output:
            (conv_module, aux_module, out_shape)
        """
        
        conv = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)
        dropout = nn.Dropout3d(p=dropout_prob)
        conv_module = nn.Sequential(conv, dropout).to(self.device_)
        out_shape = conv_output_shape(in_shape, kernel_size, stride)
        
        flatten = nn.Flatten(start_dim=2)
        aux = nn.Linear(in_features=np.prod(out_shape[-2:])*out_channels, out_features=128)
        aux_module = nn.Sequential(flatten, aux).to(self.device_)
        
        return conv_module, aux_module, out_shape
    
    def _linear_unit(self, in_features, out_features, dropout_prob):
        relu = nn.ReLU()
        linear = nn.Linear(in_features, out_features)
        dropout = nn.Dropout(p=dropout_prob)
        return nn.Sequential(relu, linear, dropout).to(self.device_), out_features
        
    
    def __init__(self, in_height, in_width, seq_len, dropout_prob=0.5, aux_history=10, device=None):
        r""" TemporalCNN: this model does 3D convolution on the H, W and temporal dimension
             It also includes residual connection from each Conv3D output to the final output
             
             Args:
                 in_height: image height
                 in_width: image width
                 seq_len: image sequence length
                 dropout_prob: prob for the dropout layer
                 aux_history: length of history to extract from seq_len
             
             Output:
                 nn.Module, accepts input with shape [batch_len, seq_len, C, in_height, in_width]
        """
        super(TemporalCNN, self).__init__()
        
        self.seq_len_ = seq_len
        self.in_width_ = in_width
        self.in_height_ = in_height
        self.aux_history_ = aux_history
        in_shape = (seq_len, in_height, in_width)
        
        self.device_ = device if device is not None else torch.device("cpu")
        
        # conv1
        self.conv0_, self.aux0_, out_shape = self._conv_unit(3, 64, in_shape, (3, 12, 12), (1, 6, 6), dropout_prob)
        # conv2
        self.conv1_, self.aux1_, out_shape = self._conv_unit(64, 64, out_shape, (2, 5, 5), (1, 2, 2), dropout_prob)
        # conv3
        self.conv2_, self.aux2_, out_shape = self._conv_unit(64, 64, out_shape, (2, 5, 5), (1, 1, 1), dropout_prob)
        # conv4
        self.conv3_, self.aux3_, out_shape = self._conv_unit(64, 64, out_shape, (2, 5, 5), (1, 1, 1), dropout_prob)
        
        # Flatten the last 3 dims
        self.flatten_ = nn.Flatten(start_dim=2).to(self.device_)
        
        # FC 1024
        self.linear0_, out_features = self._linear_unit(64*np.prod(out_shape[-2:]), 1024, dropout_prob)
        # FC 512
        self.linear1_, out_features = self._linear_unit(out_features, 512, dropout_prob)
        # FC 256
        self.linear2_, out_features = self._linear_unit(out_features, 256, dropout_prob)
        # FC 128
        self.linear3_ = nn.Linear(out_features, 128).to(device)
        
        self.elu_ = nn.ELU().to(self.device_)
        
    def forward(self, x):
        x = x.permute([0, 2, 1, 3, 4]) # swap channel and seq_len, 3D conv over seq_len as depth channel
                                       # now: [batch_size, channel, seq_len, H, W]
        
        aux_outputs = []
        for layer in [(self.conv0_, self.aux0_),
                      (self.conv1_, self.aux1_),
                      (self.conv2_, self.aux2_),
                      (self.conv3_, self.aux3_)]:
            x_out = layer[0](x)
            x_out_permuted = x_out.permute([0, 2, 1, 3, 4]) # swap back for calculation of aux output
            x_aux = layer[1](x_out_permuted[:,-self.aux_history_:,:,:,:])
#             print(x_out.shape)
#             print(x_aux.shape)
            aux_outputs.append(x_aux)
            x = x_out
        
        x = x.permute([0, 2, 1, 3, 4]) # swap back the dimensions, now: [batch_size, seq_len, channel, H, W]
        
        for layer in [self.flatten_, self.linear0_, self.linear1_, self.linear2_, self.linear3_]:
            x = layer(x)
        
        final_out = x
        for aux_out in aux_outputs:
            final_out = final_out + aux_out
        final_out = self.elu_(final_out)
#         print(final_out.shape)
        
        return final_out


## Autoregressive LSTM

In [5]:
# Autoregressive LSTM Module

class AutoregressiveLSTM(nn.Module):
    
    def __init__(self, target_size, visual_feature_size, hidden_size, internal_lstm, autoregressive_mode=True, device=None):
        super(AutoregressiveLSTM, self).__init__()
        
        if device is None:
            device = torch.device("cpu")
        self.device_ = device
        
        self.autoregressive_mode_ = autoregressive_mode
        self.lstm_ = internal_lstm # LSTM module shared between Autoregressive and Groundtruth
        self.linear_ = nn.Linear(visual_feature_size + target_size + hidden_size,
                                 target_size).to(device)
        self.target_size_ = target_size
        self.visual_feature_size_ = visual_feature_size
        self.hidden_size_ = hidden_size
    
    def forward(self, visual_features, prev_target=None, prev_states=None, target_groundtruth=None):
        
        assert (self.autoregressive_mode_) or (target_groundtruth is not None)
        
        if prev_target is None or not self.autoregressive_mode_:
            batch_size, seq_len, _ = visual_features.shape
            prev_target = torch.zeros(batch_size, seq_len, self.target_size_, device=self.device_)
            
        if self.autoregressive_mode_:
            lstm_input = torch.cat((visual_features, prev_target), dim=-1)
        else:
            lstm_input = torch.cat((visual_features, target_groundtruth), dim=-1)
    
        lstm_output, lstm_states = self.lstm_(lstm_input, prev_states)
        del lstm_input
        
        linear_input = torch.cat((visual_features, prev_target, lstm_output), dim=-1)
        linear_output = self.linear_(linear_input)
        del linear_input
        
        return (linear_output, lstm_states)
        

In [6]:
class SteerNet(nn.Module):
    
    def __init__(self, device=None, dropout_prob=0.0):
        super(SteerNet, self).__init__()
        
        self.device_ = device if device is not None else torch.device("cpu")
        
        self.model_cnn_ = TemporalCNN(in_height=480, in_width=640, seq_len=15, dropout_prob=dropout_prob, aux_history=10, device=device)
        
#         # Freeze CNN
#         for p in self.model_cnn_.parameters():
#             p.requires_grad = False
            
        self.feature_dropout_ = nn.Dropout(p=dropout_prob)
        self.lstm_ = nn.LSTM(input_size=128+3, hidden_size=64, batch_first=True).to(device)
        self.model_lstm_gt_ = AutoregressiveLSTM(target_size=3, visual_feature_size=128, hidden_size=64, internal_lstm=self.lstm_, autoregressive_mode=False, device=device)
        self.model_lstm_autoreg_ = AutoregressiveLSTM(target_size=3, visual_feature_size=128, hidden_size=64, internal_lstm=self.lstm_, autoregressive_mode=True, device=device)
        
        self.lstm_state_ = None
        self.preserve_states_ = False
        
        self.train_ = True
        
        # for testing
        self.test_lstm_ = nn.LSTM(input_size=128, hidden_size=64, batch_first=True, num_layers=2).to(device)
        self.test_lstm_linear_ = nn.Linear(64, 3).to(device)
    
    def forward(self, images, target=None):
        assert target is None or self.train_==True
        
        features = self.model_cnn_(images)
        features = self.feature_dropout_(features)
        
#         if self.preserve_states_:
            
#             if self.lstm_state_ is None:
#                 if self.train_:
#                     out_gt, state_gt = self.model_lstm_gt_(features, target_groundtruth=target)
#                 out_autoreg, state_autoreg = self.model_lstm_autoreg_(features)
                
#             else:
#                 state_autoreg = self.lstm_state_[0]
#                 if self.train_:
#                     state_gt = self.lstm_state_[1]
#                     out_gt, state_gt = self.model_lstm_gt_(features, prev_states=state_gt, target_groundtruth=target)
#                 out_autoreg, state_autoreg = self.model_lstm_autoreg_(features, prev_states=state_autoreg)
                
#             state_autoreg_next = (state_autoreg[0].clone().detach(), state_autoreg[1].clone().detach())
#             if self.train_:
#                 state_gt_next = (state_gt[0].clone().detach(), state_gt[1].clone().detach())
#             else:
#                 state_gt_next = None
                
#             self.lstm_state_ = (state_autoreg_next, state_gt_next)
        
#         else:
#             if self.train_:
#                 out_gt, _ = self.model_lstm_gt_(features, target_groundtruth=target)
#             else:
#                 out_autoreg, _ = self.model_lstm_autoreg_(features)

        # for testing
#         out_autoreg = self.test_linear_(features)
#         out_gt = self.test_linear_(features)
        out, state = self.test_lstm_(features)
        out_autoreg = self.test_lstm_linear_(out)
        out_gt = out_autoreg
        
        return (out_autoreg, out_gt) if self.train_ else out_autoreg
    
    def train(self, mode=True):
        self.train_ = mode
        
    def preserve_states(self, mode=False):
        self.preserve_states_ = mode

# Training

In [7]:
from torch.utils.data import BatchSampler, SequentialSampler, RandomSampler
# train - validation split

udacity_dataset = UdacityDataset(csv_file='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/interpolated.csv',
                              root_dir='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/',
                              transform=transforms.Compose([transforms.ToTensor()]),
                              select_camera='center_camera')

dataset_size = int(len(udacity_dataset) * 0.10)
split_point = int(dataset_size * 0.8)
training_set = UdacityDataset(csv_file='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/interpolated.csv',
                              root_dir='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/',
                              transform=transforms.Compose([transforms.ToTensor()]),
                              select_camera='center_camera',
                              select_range=(0, split_point))
validation_set = UdacityDataset(csv_file='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/interpolated.csv',
                                root_dir='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/',
                                transform=transforms.Compose([transforms.ToTensor()]),
                                select_camera='center_camera',
                                select_range=(split_point, dataset_size))

print("size of training set: {}\nstat:\nmean: {}\nstd: {}".format(len(training_set), training_set.mean, training_set.std))
print("size of validation set: {}\nstat:\nmean: {}\nstd: {}".format(len(validation_set), validation_set.mean, validation_set.std))

cbs_train = ConsecutiveBatchSampler(data_source=training_set, batch_size=10, shuffle=True, drop_last=True, seq_len=15)
cbs_valid = ConsecutiveBatchSampler(data_source=validation_set, batch_size=10, shuffle=True, drop_last=True, seq_len=15)

trainig_loader = DataLoader(training_set, sampler=cbs_train, collate_fn=(lambda x: x[0]))
validation_loader = DataLoader(validation_set, sampler=cbs_valid, collate_fn=(lambda x: x[0]))

size of training set: 8111
stat:
mean: {'angle': -0.008475752983277953, 'torque': -0.09077343871020742, 'speed': 15.622631789553342}
std: {'angle': 0.27155946576081297, 'torque': 0.7871942194690468, 'speed': 5.695705986851927}
size of validation set: 2028
stat:
mean: {'angle': -0.008475752983277953, 'torque': -0.09077343871020742, 'speed': 15.622631789553342}
std: {'angle': 0.27155946576081297, 'torque': 0.7871942194690468, 'speed': 5.695705986851927}


In [8]:
import datetime
import subprocess
from torch.utils.tensorboard import SummaryWriter

from torchviz import make_dot

writer = SummaryWriter('/export/jupyterlab/chengxin/runs/steernet-{}'.format(datetime.datetime.now()))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} for training".format(device))

steernet = SteerNet(device=device, dropout_prob=0.75)
steernet.preserve_states(True)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(steernet.parameters(), lr=1e-4)

def normalized_data(sample, channel, dataset_for_stat, seq_len=10):
    result = (sample[channel][:, -seq_len:] - dataset_for_stat.mean[channel]) / dataset_for_stat.std[channel]
    return result

def do_epoch(epoch_num=0):
    
    # training
    steernet.train()
    
    for i, sample in enumerate(trainig_loader):
        
        angle = sample['angle'][:, -10:]
        torque = sample['torque'][:, -10:]
        speed = sample['speed'][:, -10:]
        images = sample['image'].to(device)
#         angle = normalized_data(sample, 'angle', training_set, seq_len=10)
#         torque = normalized_data(sample, 'torque', training_set, seq_len=10)
#         speed = normalized_data(sample, 'speed', training_set, seq_len=10)
        
        target = torch.stack([angle, torque, speed])
        target = target.permute([2, 1, 0]).to(device) # (batch_size x seq_len x target_dim)
        angle = angle.to(device)
        del sample, torque, speed
        
        optimizer.zero_grad()
        
        out_autoreg, out_gt = steernet(images, target)
        out_autoreg_angle = out_autoreg[:, :, 0]
        
        loss_angle = criterion(angle, out_autoreg_angle)
        loss_autoreg = criterion(out_autoreg, target)
        loss_gt = criterion(out_gt, target)
        
        loss_total = loss_angle + do_epoch.loss_target_weight * (loss_autoreg + loss_gt)
        loss_total.backward()
        
        writer.add_scalar('train - loss total epoch {}'.format(epoch_num), loss_total.item(), i)
        writer.add_scalar('train - loss angle epoch {}'.format(epoch_num), loss_angle.item(), i)
        writer.add_scalar('train - loss autoreg epoch {}'.format(epoch_num), loss_autoreg.item(), i)
        writer.add_scalar('train - loss gt epoch {}'.format(epoch_num), loss_gt.item(), i)
        
#         torch.nn.utils.clip_grad_norm_(steernet.parameters(), 15.0) # gradient clamping
        optimizer.step()
        
    print("Finished training epoch {}".format(epoch_num))
    
    git_label = subprocess.check_output(["git", "rev-parse", "--short=6", "HEAD"]).strip().decode('UTF-8') # get current git label
    torch.save(steernet.state_dict(), "/export/jupyterlab/chengxin/models/git-{}-epoch-{}".format(git_label, epoch_num))
    
    # validation
    
    with torch.no_grad(): # we don't require grad calcualation in the validation phase. it saves memory.
        steernet.eval()

        running_cost = 0.0
        total_iterations = 0
        for i, sample in enumerate(validation_loader):
            images = sample['image'].to(device)
            angle = sample['angle'][:, -10:]
            torque = sample['torque'][:, -10:]
            speed = sample['speed'][:, -10:]
#             angle = normalized_data(sample, 'angle', training_set, seq_len=10)
#             torque = normalized_data(sample, 'torque', training_set, seq_len=10)
#             speed = normalized_data(sample, 'speed', training_set, seq_len=10)
            
            target = torch.stack([angle, torque, speed])
            target = target.permute([2, 1, 0]).to(device) # (batch_size x seq_len x target_dim)
            angle = angle.to(device)
            del sample, torque, speed

            out_autoreg = steernet(images)
            out_autoreg_angle = out_autoreg[:, :, 0]

            loss_angle = criterion(angle, out_autoreg_angle)
            loss_autoreg = criterion(out_autoreg, target)
            
            writer.add_scalar('valid - loss angle epoch {}'.format(epoch_num), loss_angle.item(), i)
            writer.add_scalar('valid - loss autoreg epoch {}'.format(epoch_num), loss_autoreg.item(), i)
#             running_cost += loss_angle.item() * training_set.std['angle']**2
            running_cost += loss_angle.item()
            total_iterations = i+1
        
        running_cost = running_cost / total_iterations
        writer.add_scalar('valid - angle (RMSE)', np.sqrt(running_cost), epoch_num)
                  
    print("Finished validating epoch {}".format(epoch_num))
    
do_epoch.loss_target_weight = 0.0

Using cuda for training


In [ ]:
for epoch in range(0, 20):
    do_epoch(epoch)